In [45]:
import gzip
import json
import requests
import os
import spacy

In [ ]:
def download_kb(destination_dir="../data/natural_questions/"):
    # URL for the dataset
    url = "https://huggingface.co/MartinoMensio/spaCy-entity-linker/resolve/main/knowledge_base.tar.gz"
    
    # Make sure the destination directory exists
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Path to save the file
    save_path = os.path.join(destination_dir, "knowledge_base.tar.gz")
    
    # Download the file
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an error for failed requests
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

    print(f"Downloaded the dataset to {save_path}")
    
# download_kb()

In [69]:
data = []

# initialize language model
nlp = spacy.load("en_core_web_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

with gzip.open('../data/natural_questions/simplified-nq-train.jsonl.gz', 'rt', encoding='utf-8') as f:
    counter = 0
    for i, line in enumerate(f):
        entry = json.loads(line)
        
        short_answer = entry['annotations'][0]['short_answers']
        long_answer = entry['annotations'][0]['long_answer']['candidate_index']
        
        if (short_answer != []) and (long_answer != -1):
            query = nlp(entry['question_text'])
            linked_query_entities = query._.linkedEntities
            print(entry['question_text'])
            print(linked_query_entities.__len__())
            linked_query_entities.pretty_print()
            
            # doc = nlp(entry['document_text'])
            # linked_doc_entities = doc._.linkedEntities
            # print(entry['document_text'])
            # linked_doc_entities.pretty_print()
                        
            short_answer_text = nlp(' '.join(entry['document_text'].split(' ')[short_answer[0]['start_token']:short_answer[0]['end_token']]))
            linked_short_answer_entities = short_answer_text._.linkedEntities
            print(' '.join(entry['document_text'].split(' ')[short_answer[0]['start_token']:short_answer[0]['end_token']]))
            print(linked_short_answer_entities.__len__())
            linked_short_answer_entities.pretty_print()
            
            print('\n')
            
            
            if i > 50:
                break
            
# print(counter)

which is the most common use of opt-in e-mail marketing
2
<EntityElement: https://www.wikidata.org/wiki/Q7901733 Use                                                                         >
<EntityElement: https://www.wikidata.org/wiki/Q82435 e                         mathematical constant                             >
a newsletter sent to an advertising firm 's customers
3
<EntityElement: https://www.wikidata.org/wiki/Q264238 newsletter                regularly distributed publication                 >
<EntityElement: https://www.wikidata.org/wiki/Q852835 customer                  recipient of a good, service, product or idea     >
<EntityElement: https://www.wikidata.org/wiki/Q37038 advertising               form of communication for marketing, typically paid for>


how i.met your mother who is the mother
2
<EntityElement: https://www.wikidata.org/wiki/Q7560 mother                    female parent                                     >
<EntityElement: https://www.wikidata.org/wiki/Q7